## Model Building

Build logistic regression, random forest, and XGBoost models to predict the probability of an upset in the NCAA tournament

Based on data output in M1_DataCleaning.ipynb, model used in M3_Predictions.py

In [1]:
import pandas as pd
import random
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tqdm import tqdm
import pickle
import numpy as np

In [2]:
# matchup data outputted from M1_DataCleaning.ipynb
matchups = pd.read_csv('mydata/mens/matchups.csv')
matchups.head()

,Season,TeamrankRating_x,TrankRating_x,OE_x,DE_x,Tempo_x,Seed_x,3ptRate_x,Ast%_x,FT%_x,...,AbsyOffxDefAstDiff,xOffyDefAstAvg,yOffxDefAstAvg,xOffyOffAstDiff,TotalPossVarSum,GameScoreVarSum,TrankNaiveUpsetProbability,TeamrankNaiveUpsetProbability,Upset,ScorePerPossDiff
0,2008.0,28.5,0.960742,117.213494,88.761128,73.7,2,0.390996,0.522826,0.694618,...,0.152258,0.528512,0.552607,-0.024095,342.200873,0.030100,0.011556,0.022079,0,0.014302
1,2008.0,28.5,0.960742,117.213494,88.761128,73.7,2,0.390996,0.522826,0.694618,...,0.109056,0.520698,0.531006,-0.010308,295.048054,0.047594,0.312558,0.323143,1,-0.085861
2,2008.0,23.6,0.927155,115.187217,92.329124,65.8,3,0.371802,0.610714,0.750341,...,0.018604,0.564642,0.594836,-0.030194,269.327900,0.072884,0.459461,0.493502,0,0.059648
3,2008.0,32.4,0.981585,120.970641,85.610492,69.5,1,0.291796,0.627572,0.707756,...,0.078373,0.587891,0.574597,0.013293,266.467752,0.058420,0.030979,0.039768,0,0.375704
4,2008.0,32.4,0.981585,120.970641,85.610492,69.5,1,0.291796,0.627572,0.707756,...,0.071639,0.587224,0.571230,0.015993,309.871774,0.046553,0.084577,0.069552,0,0.310661


In [3]:
y = matchups['Upset']
X = matchups.drop(columns = ['Upset', 'ScorePerPossDiff'])

In [4]:
myCViterator = []
for i in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
    trainIndices = X[X['Season'] != i].index.values.astype(int)
    testIndices = X[X['Season'] == i].index.values.astype(int)
    myCViterator.append( (trainIndices, testIndices) )

In [5]:
log_loss(y, X['TrankNaiveUpsetProbability'])

0.5380199783067582

In [6]:
log_loss(y, (0.6 * X['TeamrankNaiveUpsetProbability'] + 0.4 * X['TrankNaiveUpsetProbability']))

0.5404347706678986

In [7]:
log_loss(y, [sum(y) / len(y) for i in range(len(y))])

0.6039235957046507

### Logistic Regression

In [9]:
# lasso regularization for feature selection
# coefficients range from 0.02 to ~2000
# each coefficient is 3% larger than last, there are 400 coefficients in total
lasso_reg = [0.02 * (1.02) ** i for i in range(200)]

# ridge regularization to prevent overfitting
ridge_reg = [1e-2, 3e-2, 1e-1, 3e-1 , 1e0, 3e0]

# scale used to scale columns before applying regularization
scale = StandardScaler()

def train_logistic_regression(features = None, regularization = True):
    
    # list to store results
    feature_list = []
    scores = []
    ridge_coefs = []
    models = []
    num_features = []
    
    # for each possible lasso regularization coefficient
    for c in tqdm(lasso_reg):
        
        # dont use season as a feature, use inputted features if given
        if features:
            X_lasso = X[features]
        else:
            X_lasso = X.drop(columns = ['Season'])
            
        if regularization:
        
            # scale columns before regularization
            X_lasso = pd.DataFrame(scale.fit_transform(X_lasso), columns = X_lasso.columns)

            # fit L1 logistic regression for feature selection
            lasso = LogisticRegression(penalty = 'l1', C = c, solver = 'saga', max_iter = 10000).fit(X_lasso, y)

            # filter for the columns with nonzero coefficients and the season column
            zero_cols = ['Season']
            if features:
                zero_cols = list(set(X.columns) - set(features))
            for i in range(len(lasso.coef_[0])):
                if lasso.coef_[0][i] == 0.0:
                    zero_cols.append(X_lasso.columns[i])
            nonzero_X = X.drop(columns = zero_cols)
        
        else:
            if features:
                nonzero_X = X.copy()[features]
            else:
                nonzero_x = X.drop(columns = ['Season'])
            
        
        # if there is at least 1 nonzero column and the same amount of features hasn't already been built
        if (len(nonzero_X.columns) > 0 and (len(nonzero_X.columns)) not in num_features):
            
                    
            # retrain model on full dataset for coefficients
            log_model = LogisticRegressionCV(penalty = 'l2', 
                                             Cs = ridge_reg, 
                                             max_iter = 10000, 
                                             random_state = 0, 
                                             solver = "sag",
                                             scoring = 'neg_log_loss',
                                             cv = myCViterator)
            log_fit = log_model.fit(pd.DataFrame(scale.fit_transform(nonzero_X), columns = nonzero_X.columns), y)

            # store model details
            feature_list.append(nonzero_X.columns)
            num_features.append(len(nonzero_X.columns))
            models.append(log_fit)
            ridge_coefs.append(log_fit.C_)
            scores.append(np.mean(log_fit.scores_[1][:, ridge_reg.index(log_fit.C_)]))
                
    # return dataframe of results
    return pd.DataFrame({'Type': ['log' for i in range(len(models))],
                         'Num_Features': num_features,
                         'Features': feature_list,
                         'Model': models,
                         'Model_Coef': ridge_coefs,
                         'Avg_Score': scores})

### Random Forest

In [ ]:
# random forest paramaters
max_features_list = [5, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 35, 40]
rf_max_depths = [5, 10, 15, 20, None]
min_samples = [1, 0.01, 0.03, 0.1]
criterions = ['gini', 'entropy']

def train_random_forest():
    
    # list/dictionaries to store results
    scores = {}
    models = []
    model_coefs = []
    
    # for repeatable randomization
    random.seed(0)
    
    # fit 1 models
    for i in tqdm(range(50)):
        
        # randomly generate random forest params
        rf_params = {'max_features': max_features_list[random.randint(0, len(max_features_list) - 1)],
                     'max_depth': rf_max_depths[random.randint(0, len(rf_max_depths) - 1)],
                     'min_samples_leaf': min_samples[random.randint(0, len(min_samples) - 1)],
                     'criterion': criterions[random.randint(0, len(criterions) - 1)]
                    }
        
        # keep generating params if they've already been tested
        while rf_params in list(prev_results['Model_Coef']):
            rf_params = {'max_features': max_features_list[random.randint(0, len(max_features_list) - 1)],
                     'max_depth': rf_max_depths[random.randint(0, len(rf_max_depths) - 1)],
                     'min_samples_leaf': min_samples[random.randint(0, len(min_samples) - 1)],
                     'criterion': criterions[random.randint(0, len(criterions) - 1)]
                    }
        
        # cross validate over each season
        for season in list(X['Season'].unique()):
            
            # add season to scores dictionary
            if season not in scores:
                scores[season] = []
            
            # split into train and validation sets
            X_train = X[X['Season'] != season].drop(columns = ['Season'])
            X_val = X[X['Season'] == season].drop(columns = ['Season'])
            y_train = y[X_train.index]
            y_val = y[X_val.index]
            
            # fit random forest model
            rf_model = RandomForestClassifier(n_estimators = 500,
                                             criterion = rf_params['criterion'],
                                             max_depth = rf_params['max_depth'],
                                             min_samples_leaf = rf_params['min_samples_leaf'],
                                             max_features = rf_params['max_features'],
                                             random_state = 0).fit(X_train, y_train)
            
            # predict win probabilities
            predictions = rf_model.predict_proba(X_val)
            
            # calculate log loss and store score
            val_score = log_loss(y_val, predictions)
            scores[season].append(val_score)
            
        # retrain model on full dataset for feature importances
        rf_model = RandomForestClassifier(n_estimators = 1000,
                                         criterion = rf_params['criterion'],
                                         max_depth = rf_params['max_depth'],
                                         min_samples_leaf = rf_params['min_samples_leaf'],
                                         max_features = rf_params['max_features'],
                                         random_state = 0).fit(X.drop(columns = ['Season']), y)
        
        # store model details
        model_coefs.append(rf_params)
        models.append(rf_model)
        
    # return dataframe of results
    return pd.DataFrame({'Type': ['rf' for i in range(len(models))],
                         'Num_Features': ['All' for i in range(len(models))],
                         'Features': ['All' for i in range(len(models))],
                         'Model': models,
                         'Model_Coef': model_coefs,
                         '2008_Score': scores[2008],
                         '2009_Score': scores[2009],
                         '2010_Score': scores[2010],
                         '2011_Score': scores[2011],
                         '2012_Score': scores[2012],
                         '2013_Score': scores[2013],
                         '2014_Score': scores[2014],
                         '2015_Score': scores[2015],
                         '2016_Score': scores[2016],
                         '2017_Score': scores[2017],
                         '2018_Score': scores[2018],
                         '2019_Score': scores[2019]})

### XGBoost

In [ ]:
# XGBoost parameters
etas = [0.003, 0.01, 0.03, 0.1, 0.3, 1, 3]
xgb_max_depths = [2, 6, 10, 14, 18]
min_child_weights = [0, 1, 3, 6, 10, 14]
gammas = [0, 0.01, 0.03, 0.1, 0.3, 1, 3]
subsamples = [0.8, 0.9, 1]
lambdas = [0.003,0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300]

def train_xgboost():
    
    # list/dic to store results
    scores = []
    models = []
    model_coefs = []
    
    # for repeatable randomization
    random.seed(0)
    
    # fit 1 models
    for i in tqdm(range(1)):
        
        # randomly generate XGBoost params
        xgb_params = {'eta': etas[random.randint(0, len(etas) - 1)],
                      'max_depth': xgb_max_depths[random.randint(0, len(xgb_max_depths) - 1)],
                      'min_child_weight': min_child_weights[random.randint(0, len(min_child_weights) - 1)],
                      'gamma': gammas[random.randint(0, len(gammas) - 1)],
                      'subsample': subsamples[random.randint(0, len(subsamples) - 1)],
                      'lambda': lambdas[random.randint(0, len(lambdas) - 1)],
                      'random_state': 0
                     }
        # keep randomly generating if params have already been tested
        while xgb_params in list(prev_results['Model_Coef']):
            # randomly generate XGBoost params
            xgb_params = {'eta': etas[random.randint(0, len(etas) - 1)],
                          'max_depth': xgb_max_depths[random.randint(0, len(xgb_max_depths) - 1)],
                          'min_child_weight': min_child_weights[random.randint(0, len(min_child_weights) - 1)],
                          'gamma': gammas[random.randint(0, len(gammas) - 1)],
                          'subsample': subsamples[random.randint(0, len(subsamples) - 1)],
                          'lambda': lambdas[random.randint(0, len(lambdas) - 1)],
                          'random_state': 0
                         }
        
        # cross validate over each season
        for season in list(X['Season'].unique()):
            
            # add season to scores dictionary
            if season not in scores:
                scores[season] = []
                
            # split into train and validation sets
            X_train = X[X['Season'] != season].drop(columns = ['Season'])
            X_val = X[X['Season'] == season].drop(columns = ['Season'])
            y_train = y[X_train.index]
            y_val = y[X_val.index]
            
            # fit XGBoost model
            xgb_model = XGBClassifier(**xgb_params).fit(X_train, y_train)
            
            # predict win probabilities
            predictions = xgb_model.predict_proba(X_val)
            
            # calculate log loss and store score
            val_score = log_loss(y_val, predictions)
            scores[season].append(val_score)
            
        # retrain model on full dataset for feature importances
        xgb_model = XGBClassifier(**xgb_params).fit(X.drop(columns = ['Season']), y)
        
        # store model details
        model_coefs.append(xgb_params)
        models.append(xgb_model)
        
    # return dataframe of results
    return pd.DataFrame({'Type': ['xgb' for i in range(len(models))],
                         'Num_Features': ['All' for i in range(len(models))],
                         'Features': ['All' for i in range(len(models))],
                         'Model': models,
                         'Model_Coef': model_coefs,
                         '2008_Score': scores[2008],
                         '2009_Score': scores[2009],
                         '2010_Score': scores[2010],
                         '2011_Score': scores[2011],
                         '2012_Score': scores[2012],
                         '2013_Score': scores[2013],
                         '2014_Score': scores[2014],
                         '2015_Score': scores[2015],
                         '2016_Score': scores[2016],
                         '2017_Score': scores[2017],
                         '2018_Score': scores[2018],
                         '2019_Score': scores[2019]})

### Results

In [11]:
# train the models
best_features = ['TrankRating_x', 'ORB%_x', 'FTRVar_x', 'OR%Var_y', 'SeedDiff',
                   'TrankPredictedSpread', 'xOffyOffTODiff', 'xOffyDefAstAvg',
                   'TrankNaiveUpsetProbability', 'TeamrankNaiveUpsetProbability', 
                 'TeamrankPredictedSpread', '3ptRate_y', '3ptRate_x', 'xOffyOffRebDiff',
                'xOffyOffEFGDiff', 'TrankPredictedPoss', 'OppFT%_x', 'OppFT%_y', '3pt%Var_y',
                'TeamrankRating_x', 'OppAst%_x', 'TORD%_x',
               'FTRD_x', 'Opp3ptRateVar_x', 'OppAst%Var_x', 
               'OppFTRVar_x', 'DE_y', 'Tempo_y', '3P%D_y',  'OppAst%Var_y',
               'FTRVar_y', 'OR%Var_y', 'TotalPossVar_y', 
               'TrankTempoDiff',
               'xOffyDefFTRateAvg',  'TotalPossVarSum']
small_features = ['TrankRating_x', 'SeedDiff', 'TrankRating_y', 'TeamrankRating_x', 'TeamrankRating_y',
                   'TrankPredictedSpread', 'xOffyOffTODiff', 'xOffyDefAstAvg',
                   'TrankNaiveUpsetProbability', 'TeamrankNaiveUpsetProbability', 
                 'TeamrankPredictedSpread', 'xOffyOffRebDiff']
chosen_features = ['TeamrankRating_x', 'TrankRating_x', 'ORB%_x', 'FTRVar_x', 'DE_y',
       '3P%D_y', '3pt%Var_y', 'FTRVar_y', 'OR%Var_y', 'TotalPossVar_y',
       'TrankPredictedSpread', 'xOffyOffTODiff', 'xOffyDefAstAvg',
       'TrankNaiveUpsetProbability', 'TeamrankNaiveUpsetProbability', 'SeedDiff']
#log_results = train_logistic_regression()
#log_results2 = train_logistic_regression(best_features)
log_results4 = train_logistic_regression(small_features)
log_results3 = train_logistic_regression(chosen_features, False)
final_results = pd.concat([log_results, log_results2, log_results3, log_results4], ignore_index = True)
#rf_results = train_random_forest()
#xgb_results = train_xgboost()
#final_results = pd.concat([prev_results, log_results, rf_results, xgb_results], ignore_index = True)

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 315.28it/s]


In [12]:
# function to show feature importances of given model by results row index
def features(row_index):
    model = final_results.loc[row_index, 'Model']
    
    # if model is logistic regression, return coefficients
    if final_results.loc[row_index, 'Type'] == 'log':
        model_features = list(final_results.loc[row_index, 'Features'])
        sign_list = []
        coef_list = []
        for i in range(len(model_features)):
            coef = round(model.coef_[0][i], 5)
            coef_list.append(coef)
            if coef > 0:
                sign_list.append('+')
            else:
                sign_list.append('-')
        
        return_df =  pd.DataFrame({'Feature': model_features,
                             'Sign': sign_list,
                            'Coefficient': coef_list})
        return_df = pd.concat([return_df, pd.DataFrame({'Feature': ['Intercept'],
                                                      'Sign': ['-'],
                                                      'Coefficient': [model.intercept_]})], ignore_index = True)
        return return_df.reindex(return_df['Coefficient'].abs().sort_values(ascending = False).index)
        
    # if model is random forest or XGBoost, return feature importance plot
    else:
        feat_importances = pd.Series(model.feature_importances_, index = X.drop(columns = ['Season']).columns)
        return feat_importances.nlargest(15).plot(kind = 'barh')

In [13]:
# function to show params of given model by results row index
def params(row_index):
    return final_results.loc[row_index, 'Model_Coef']

In [14]:
# print results
final_results[['Type', 'Num_Features', 'Avg_Score']].sort_values(by = ['Avg_Score'], ascending = False).head(60)

,Type,Num_Features,Avg_Score
75,log,15,-0.518458
12,log,15,-0.518458
15,log,18,-0.518756
14,log,17,-0.518881
11,log,14,-0.518957
74,log,14,-0.518957
76,log,16,-0.519008
13,log,16,-0.519008
93,log,16,-0.519236
77,log,18,-0.519298


In [15]:
final_results.tail()

,Type,Num_Features,Features,Model,Model_Coef,Avg_Score
98,log,6,"Index(['TeamrankRating_x', 'TrankPredictedSpre...","LogisticRegressionCV(Cs=[0.01, 0.03, 0.1, 0.3,...",[0.1],-0.533832
99,log,7,"Index(['TeamrankRating_x', 'TrankPredictedSpre...","LogisticRegressionCV(Cs=[0.01, 0.03, 0.1, 0.3,...",[0.03],-0.533802
100,log,8,"Index(['TeamrankRating_x', 'SeedDiff', 'TrankP...","LogisticRegressionCV(Cs=[0.01, 0.03, 0.1, 0.3,...",[0.03],-0.534499
101,log,9,"Index(['TeamrankRating_x', 'TrankRating_x', 'S...","LogisticRegressionCV(Cs=[0.01, 0.03, 0.1, 0.3,...",[0.03],-0.535100
102,log,10,"Index(['TeamrankRating_x', 'TrankRating_x', 'T...","LogisticRegressionCV(Cs=[0.01, 0.03, 0.1, 0.3,...",[0.03],-0.535114


In [25]:
features(112)

,Feature,Sign,Coefficient
16,Intercept,-,[-1.117706268801151]
12,xOffyDefAstAvg,+,0.25116
8,OR%Var_y,+,0.24576
10,TrankPredictedSpread,-,-0.23802
13,TrankNaiveUpsetProbability,+,0.23391
3,FTRVar_x,+,0.18508
6,3pt%Var_y,-,-0.18405
4,DE_y,-,-0.17037
14,TeamrankNaiveUpsetProbability,+,0.16929
9,TotalPossVar_y,-,-0.16631


In [26]:
final_results.loc[112, 'Model_Coef']

array([0.03])

In [27]:
final_results.loc[112, 'Features']

Index(['TeamrankRating_x', 'TrankRating_x', 'ORB%_x', 'FTRVar_x', 'DE_y',
       '3P%D_y', '3pt%Var_y', 'FTRVar_y', 'OR%Var_y', 'TotalPossVar_y',
       'TrankPredictedSpread', 'xOffyOffTODiff', 'xOffyDefAstAvg',
       'TrankNaiveUpsetProbability', 'TeamrankNaiveUpsetProbability',
       'SeedDiff'],
      dtype='object')

In [28]:
pickle.dump(final_results.loc[112, 'Model'], open('models/MProb.sav', 'wb'))